# The NVIDIA Collectives Communications Library (NCCL)

## Introduction

NCCL (pronounced “Nickel”) is a library of multi-GPU collective and point-to-point communication primitives that are topology-aware and are designed to be light-weight, depending only on the standard C++ and CUDA libraries. NCCL can be deployed in single-process or multi-process applications, handling required inter-process communication transparently. Moreover, the API is quite similar to MPI and provides the functionality of most-used MPI primitives.

In general, NCCL is optimized for high bandwidth and low latency over PCIe and NVLink high speed interconnect for intra-node communication and sockets and InfiniBand for inter-node communication. NCCL—allows CUDA applications and DL frameworks in particular—to efficiently use multiple GPUs without having to implement complex communication algorithms and adapt them to every platform.

**Relevance to Deep Learning:** The NVIDIA AI libraries in CUDA-X depend on NCCL to provide a programming abstraction that is highly tuned for each platform and topology-aware through advanced topology detection, generic path search, and algorithms optimized for NVIDIA architectures. Consequently, developers using deep learning frameworks can rely on NCCL’s highly optimized, MPI compatible and topology aware routines, to take full advantage of all available GPUs within and across multiple nodes.

### NCCL compared with CUDA-aware MPI

Here are some differenciating factors of NCCL compared to CUDA-aware MPI:

* NCCL APIs are initiated from the CPU, but they execute on the GPU and they move or exchange data among GPU memories whereas MPI executes entirely on CPU. 
* It also uses CUDA Stream semantics with a stream parameter while MPI (CUDA-aware or otherwise) is not stream-aware.
* NCCL requires a parent communication framework like MPI or SHMEM. 
* Unlike MPI, it does not have tags.
* NCCL is most optimized for collective communication and is more efficient than MPI in dense systems like DGX.

### Architecture Overview 

Here's an overview of NCCL's architecture:

![nccl_architecture](../../images/nccl_architecture.png)

NCCL maintains separate threads in GPU and CPU for intra-node and network-bound communication, respectively. Crucially, it detects complex multi-node and intra-node topologies at runtime and generates optimal paths for data transfer between two GPUs dynamically.

For example, recall that within a DGX-1V node, device-to-device `cudaMemcpy` between GPU 0 and GPU 7 requires data movement through PCIe bus and SMP interconnect, both of which are bandwidth constrained compared to NVLink. This route is denoted by the blue path below.

![nccl_dgx1_topology](../../images/nccl_dgx1_topology.png)

Note that there isn't a direct NVLink connection between GPUs 0 and 7 on DGX with 8 V100 GPUs. NCCL, however, can utilize GPU 4 to establish a one-hop NVLink-based connection between GPU 0 and 7 as deonted by the red path above. NCCL employs many such optimizations transparently to the users that ultimately results in higher network utilization and application performance.

## Using NCCL

### Communicator Initialization

The NCCL API closely follows MPI. Before performing data transfer operations, a communicator object must be created for each GPU. The communicators identify the set of GPUs that will communicate and maps the communication paths between them. We call the set of associated communicators a "clique". The most general method to initialize communicators in NCCL is to call `ncclCommInitRank()` once for each GPU:

```c
ncclComm_t nccl_comm;
NCCL_CALL(ncclCommInitRank(&nccl_comm, nGPUs, nccl_uid, rank));
```

This function assumes that the GPU belonging to the specified rank has already been selected using `cudaSetDevice()`. `nGPUs` is the number of GPUs in the clique. `nccl_uid` allows the ranks of the clique to find each other. The same `nccl_uid` must be used by all ranks. To achieve this, call `ncclGetUniqueId()` in one rank and broadcast the resulting `nccl_uid` to the other ranks of the clique using MPI as follows:

```c
ncclUniqueId nccl_uid;
if (rank == 0) {
    ncclGetUniqueId(&nccl_uid);
}
MPI_Bcast(&nccl_uid, sizeof(ncclUniqueId), MPI_BYTE, 0, MPI_COMM_WORLD);
```

The last argument to `ncclCommInitRank()`, `rank`, specifies the index of the current GPU within the clique. It must be unique for each rank in the clique and in the range $[0, nGPUs)$.

Internally, `ncclInitRank()` performs a synchronization between all communicators in the clique. As a consequence, it must be called from a different host thread for each GPU, or from separate processes (e.g., MPI ranks). 

Thus, in a multi-node program that uses MPI, the steps to initialize NCCL communicator are as follows:

1. Get `rank` and `size` from `MPI_Comm_rank` and `MPI_Comm_size` functions, respectively. Now, `nGPUs`, which is the total number of GPUs used, will be equal to `size`, as we are using the single-GPU per rank communication model.
2. Get the unique clique ID, `nccl_uid`, using `ncclGetUniqueId` function on rank 0 and broadcast it using `MPI_Bcast`.
3. Get the local rank, `local_rank`, of the process within a node using a local MPI communicator and `MPI_Comm_split_type`, `MPI_Comm_rank`, and `MPI_Comm_free` functions as done in previous labs.
4. Use `cudaSetDevice` to set the current GPU as `local_rank`.
5. Use `ncclCommInitRank` function to initialize NCCL communicator on all ranks.

The code for this process is as follows:

```c
MPI_Init(&argc, &argv);
int rank;
MPI_Comm_rank(MPI_COMM_WORLD, &rank);
int size;
MPI_Comm_size(MPI_COMM_WORLD, &size);

ncclUniqueId nccl_uid;
if (rank == 0) ncclGetUniqueId(&nccl_uid);
MPI_Bcast(&nccl_uid, sizeof(ncclUniqueId), MPI_BYTE, 0, MPI_COMM_WORLD);

int local_rank = -1;
{
    MPI_Comm local_comm;
    MPI_Comm_split_type(MPI_COMM_WORLD, MPI_COMM_TYPE_SHARED, rank, MPI_INFO_NULL, &local_comm);
    MPI_Comm_rank(local_comm, &local_rank);
    MPI_Comm_free(&local_comm);
}
cudaSetDevice(local_rank);

ncclComm_t nccl_comm;
ncclCommInitRank(&nccl_comm, size, nccl_uid, rank);
```

### Group Calls

Several data transfer calls can be merged together using NCCL Groups by encapsulating memory copy operations between `ncclGroupStart()` and `ncclGroupEnd()` function calls. This is needed for three purposes: managing multiple GPUs from one thread (to avoid deadlocks), aggregating communication operations to improve performance, or merging multiple send/receive point-to-point operations.

It is advisible to always encapsulate NCCL communication functions within group calls.

### API Summary

We now give an API overview and list below the essential functions like communicator creation/ destruction, commonly used point-to-point communication functions, a couple of collective communication functions, and aggregating communication functions using group calls for performance optimization. 

```bash
// Communicator creation
ncclGetUniqueId(ncclUniqueId* commId);
ncclCommInitRank(ncclComm_t* comm, int nranks, ncclUniqueId commId, int rank);

// Communicator destruction
ncclCommDestroy(ncclComm_t comm);

// Point-to-point communication
ncclSend(void* sbuff, size_t count, ncclDataType_t type, int peer, ncclComm_t comm, cudaStream_t stream);
ncclRecv(void* rbuff, size_t count, ncclDataType_t type, int peer, ncclComm_t comm, cudaStream_t stream);

// Collective communication
ncclAllReduce(void* sbuff, void* rbuff, size_t count, ncclDataType_t type, ncclRedOp_t op, ncclComm_t comm, cudaStream_t stream);
ncclBroadcast(void* sbuff, void* rbuff, size_t count, ncclDataType_t type,       int root, ncclComm_t comm, cudaStream_t stream);

// Aggregation/Composition
ncclGroupStart();
ncclGroupEnd();
```

Observe that the communication calls are quite similar in synatx to MPI calls and that they allow using a stream parameter, making them stream-aware.

## Implementation Exercise

Open the [jacobi_nccl.cpp](../../source_code/nccl/jacobi_nccl.cpp) file and understand the flow of the program. Alternatively, you can navigate to `CFD/English/C/source_code/nccl/` directory in Jupyter's file browser in the left pane. Then, click to open the `jacobi_nccl.cpp` file.

Notice how closely it resembles the [jacobi_cuda_aware_mpi.cpp](../../source_code/mpi/jacobi_cuda_aware_mpi.cpp) program that we have used in previous lab.

Also open the [Makefile](../../source_code/nccl/Makefile) and notice that we include NCCL header files in `mpicxx` build command and we link NCCL libraries by using `-lnccl` flag.

After the Jacobi device kernel computation, we will use `ncclAllReduce` function to first reduce the (square of) L2 norm in the GPUs and then transfer it to the CPU in each rank. We will use the default stream in this exercise. Recall that the default (or NULL) stream is denoted by "0" and is synchronizing for the device. However, NCCL communication calls are asynchronous and will not block the host.

Implement the following marked as `TODO`:

* Reduce the device-local L2 norm, `l2_norm_d` to the global L2 norm on each device, `l2_global_norm_d`, using `ncclAllReduce()` function. Use `ncclSum` as the reduction operation. Make sure to encapsulate this funciton call within NCCL group calls, `ncclGroupStart()` and `ncclGroupEnd()`.
* Transfer the global L2 norm from each device to the host using `cudaMemcpyAsync` function.
* Perform the first set of halo exchanges by:
  - Receiving the top halo from the `top` neighbour into the `a_new` device memory array location. 
  - Sending current device's bottom halo to `bottom` neighbour from the `a_new + (iy_end - 1) * nx` device memory array location.
* Similarily, perform the second set of halo exchanges.
* For all NCCL calls, use "0" in the stream parameter function argument.

After implementing these, compile the program:

In [ ]:
! cd ../../source_code/nccl/ && make clean && make

Ensure that there are no errors. Now, validate your implementation by running the program binary. Due to limited resources, we will be using smaller grid ($2K\times4K$) using 2 GPUs.

- **To run across 2 nodes with 16 GPUs with $16K\times32K$ grid size, use: `mpirun -np 16 --map-by ppr:4:socket ./jacobi_nccl -ny 32768`.**
- **To run on half a node with 4 GPUs with $4K\times8K$ grid size, use: `mpirun -np 4 --map-by ppr:4:socket ./jacobi_nccl -nx 4096 -ny 8192.`**

In [ ]:
# using 2 GPUs
! cd ../../source_code/nccl/ && srun --partition=gpu  --nodes=1 --gres=gpu:2  --ntasks=2 --mpi=pmix --ntasks-per-socket=2 ./jacobi_nccl -nx 2048 -ny 4096

### DGX system with 8 Ampere A100
Partial results obtained from a DGX system with 8 A100s:

Using 2 GPUs (2K$\times$4K grid size)
```bash
Num GPUs: 2.
2048x4096: 1 GPU:   0.1378 s, 2 GPUs:   0.1759 s, speedup:     0.78, efficiency:    39.17 
```

Using 2 nodes connected by InfiniBand (IB) NICs (16384$\times$32768 grid size)
```bash
Num GPUs: 16.
16384x32768: 1 GPU:   6.5850 s, 16 GPUs:   0.7195 s, speedup:     9.15, efficiency:    57.20 
```

### DGX system with 8 Tesla V100
Partial results obtained from a DGX system with 8 Tesla V100s using 2 nodes connected by InfiniBand (IB) NICs (16384$\times$32768 grid size)

```bash
Num GPUs: 16.
16384x32768: 1 GPU:   8.9160 s, 16 GPUs:   0.7863 s, speedup:    11.34, efficiency:    70.87
```

Like in MPI, the first few NCCL calls have a high overhead. Increases the number of iterations and run the program again:

In [ ]:
! cd ../../source_code/nccl/ && srun --partition=gpu  --nodes=1 --gres=gpu:2  --ntasks=2 --mpi=pmix --ntasks-per-socket=2 ./jacobi_nccl -nx 2048 -ny 4096 -niter 5000

### DGX system with 8 Ampere A100
Partial results obtained from a DGX system with 8 A100s:

Using 2 GPUs (2K$\times$4K grid size)
```bash
Num GPUs: 2.
2048x4096: 1 GPU:   0.6823 s, 2 GPUs:   0.6445 s, speedup:     1.06, efficiency:    52.94 
```

Using 2 nodes connected by InfiniBand (IB) NICs (16384$\times$32768 grid size)
```bash
Num GPUs: 16.
16384x32768: 1 GPU:  32.8925 s, 16 GPUs:   2.7484 s, speedup:    11.97, efficiency:    74.80
```

### DGX system with 8 Tesla V100
Partial results obtained from a DGX system with 8 Tesla V100s using 2 nodes connected by InfiniBand (IB) NICs (16384$\times$32768 grid size)

```bash
Num GPUs: 16.
16384x32768: 1 GPU:  44.5218 s, 16 GPUs:   3.3858 s, speedup:    13.15, efficiency:    82.18 
```

Recall that on DGX system with 8 Tesla V100, the efficiency after 5K iterations with CUDA-aware MPI was about $74\%$. NCCL is able to utilize the dense DGX-1V with 8 V100s communication topology more efficiently, resulting in better performance. Like in the MPI module labs, we can use the `-skip_single_gpu` option after validating the implementation to reduce application runtime and profiling time.

Let us now profile to learn more about NCCL optimizations.

## Profiling

Profile the application using `nsys` for 5K iterations. Skip the single-GPU run and use a $2K\times4K$ grid size.

In [ ]:
! cd ../../source_code/nccl && sbatch profiling_4g

To view the profiler report, you would need to Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../../source_code/nccl/output_profiler/jacobi_nccl_report.nsys-rep) and choosing Save Link As. Once done open the report via the GUI. On the cell output above, view the NVTX Push-Pop stats. NCCL has already been instrumented using NVTX annotations so we don't need to add our own. However, since NCCL communication calls are asynchronous with respect to the host and execute mostly on the GPU, NVTX stats are not very helpful. 

Please note the screenshot is for a larger grid size over 2 nodes.

![nccl_profiler_output](../../images/nccl_profiler_output.png)

All the NCCL communication calls have been grouped into 2 sets:

1. The `ncclAllReduce` call is displayed as the first `ncclKernel...` call in the Timeline. The latency of this call on the hardware is about $10\mu$s.
2. Both sets of halo exchanges are encapsulated in the second group which is dsiplayed as `ncclKernel_SendRecv...` in the Timeline. The latency of this call (visible in the image) is about $50\mu$s. 

The total time between two Jacobi device kernel runs is about $80\mu$s. The `cudaMemcpy` of the L2 norm back on host and the `cudaMemset` calls both take about $1-2\mu$s each which we can average to about $5\mu$ in total. Thus, the idle time between two Jacobi iterations is about $80-50-10-5=15\mu$s. Compare this with the $150\mu$s idle time in case of CUDA-aware MPI and $125\mu$s idle time in case of CUDA streams with events in a single node run. Clearly, NCCL has optimized the communication significantly.

**Note:** If you scroll in the timeline, you will observe variation in the total and idle times between Jacobi iterations due to reasons specified in the previous labs. On average, the idle time in the NCCL implementation is the least we have observed so far.

The gains are easily visible in terms of application performance. And note that the code is quite similar to CUDA-aware MPI. This illustrates the ease-of-use of NCCL. Feel free to play around by increasing number of iterations, skipping single-GPU runs for faster runtimes, using different process mappings, changing number of rocesses, and disabling P2P and GPUDirect RDMA.

**Solution:** The solution for this exercise is present in `source_code/nccl/solution` directory: [jacobi_nccl.cpp](../../source_code/nccl/solution/jacobi_nccl.cpp).

We have now learnt about using NCCL with MPI in our application. While the library performs a lot of optimizations under the hood, the API is quite simple and easy-to-use. In applications that require  lot of collective communications, like Fast Fourier Transforms and Deep Learning models, NCCL provides ample opportunity for improving performance.

Now, let us learn about the other important communication-based library that NVIDIA offers, NVSHMEM. Click below to access the next lab:

# [Next: NVSHMEM Library](../nvshmem/nvshmem.ipynb)

Here's a link to the home notebook through which all other notebooks are accessible:

# [HOME](../../../start_here.ipynb)

---
## Links and Resources

* [Concepts: Accelerating IO in data-centers](https://developer.nvidia.com/blog/accelerating-io-in-the-modern-data-center-network-io/)
* [Programming Concepts: NCCL User Guide](https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/overview.html)
* [Programming Concepts: Collective communication in NCCL](https://developer.nvidia.com/blog/fast-multi-gpu-collectives-nccl/)
* [Programming Concepts: Scaling Deep Learning programs with NCCL](https://developer.nvidia.com/blog/massively-scale-deep-learning-training-nccl-2-4/)
* [Documentation: NCCL API](https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/api.html)
* [Code: Multi-GPU Programming Models](https://github.com/NVIDIA/multi-gpu-programming-models)
* [Code: GPU Bootcamp](https://github.com/gpuhackathons-org/gpubootcamp/)

Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

## Licensing
Copyright © 2022 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.
